In [2]:
#파일 업로드
from google.colab import files
uploaded = files.upload()

Saving customer.xlsx to customer.xlsx


In [3]:
#파일 불러오기
import pandas as pd
import numpy as np
sales = pd.read_csv( './sales.csv', encoding='utf-8')
customer = pd.read_excel('./customer.xlsx')
print(sales.head())
print(customer.head())

      purchase_date item_name  item_price customer_name
0  2019-06-13 18:02       상품A       100.0           김가온
1  2019-07-13 13:05     상 품 S         NaN           김우찬
2  2019-05-11 19:42     상 품 a         NaN           김유찬
3  2019-02-12 23:40       상품Z      2600.0           김재현
4   2019-04-22 3:09       상품a         NaN           김강현
   고객이름  지역                  등록일
0  김 현성  H시  2018-01-04 00:00:00
1  김 도윤  E시                42782
2  김 지한  A시  2018-01-07 00:00:00
3  김 하윤  F시                42872
4  김 시온  E시                43127


In [7]:
# 상품명 오류 수정정
sales['item_name'] = sales['item_name'].str.upper() # 대문자로 바꾸기
sales['item_name'] = sales['item_name'].str.replace(' ', '') # 빈칸 제거하기기
pd.unique(sales['item_name'])

array(['상품A', '상품S', '상품Z', '상품V', '상품O', '상품U', '상품L', '상품C', '상품I',
       '상품R', '상품X', '상품G', '상품P', '상품Q', '상품Y', '상품N', '상품W', '상품E',
       '상품K', '상품B', '상품F', '상품D', '상품M', '상품H', '상품T', '상품J'],
      dtype=object)

In [ ]:
# 결측치 개수 확인
sales.isnull().sum()
customer.isnull().sum()

In [11]:
# 구매년월 새로 컬럼
sales['month'] = pd.to_datetime(sales['purchase_date']).dt.strftime('%Y%m')
print(sales['month'])
res = sales.pivot_table(index='month', columns='item_name', aggfunc='size', fill_value=0)
print(res)

0       201906
1       201907
2       201905
3       201902
4       201904
         ...  
2994    201902
2995    201906
2996    201903
2997    201907
2998    201907
Name: month, Length: 2999, dtype: object
item_name  상품A  상품B  상품C  상품D  상품E  상품F  상품G  상품H  상품I  상품J  ...  상품Q  상품R  \
month                                                        ...             
201901      18   13   19   17   18   15   11   16   18   17  ...   17   21   
201902      19   14   26   21   16   14   14   17   12   14  ...   22   22   
201903      17   21   20   17    9   27   14   18   12   16  ...   23   16   
201904      17   19   24   20   18   17   14   11   18   13  ...   20   20   
201905      24   14   16   14   19   18   23   15   16   11  ...   13   22   
201906      24   12   11   19   13   18   15   13   19   22  ...   15   16   
201907      20   20   17   17   12   17   19   19   19   23  ...   15   19   

item_name  상품S  상품T  상품U  상품V  상품W  상품X  상품Y  상품Z  
month                                  

In [12]:
# 고객 이름 수정
customer['고객이름'] = customer['고객이름'].str.replace(' ', '')
customer['고객이름'] = customer['고객이름'].str.replace('  ', '')
print( customer['고객이름'] )

0      김현성
1      김도윤
2      김지한
3      김하윤
4      김시온
      ... 
195    김재희
196    김도영
197    김이안
198    김시현
199    김서우
Name: 고객이름, Length: 200, dtype: object


In [14]:
#등록일 수정
date = customer['등록일'].astype('str').str.isdigit() # 시리즈
date.sum()
newDate1 = pd.to_timedelta( customer.loc[ date,'등록일'].astype('float'), unit='D') + pd.to_datetime('1900/01/01') # -2 # 엑셀과 파이썬의 날짜 형식이 달라서 -2 해 줘야 함
newDate1
newDate2 = pd.to_datetime( customer.loc[~date,'등록일'])
customer['등록일'] = pd.concat([newDate1, newDate2])
customer

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [15]:
#등록월 추출해서 등록 고객수 집계
customer['등록연월'] = customer['등록일'].dt.strftime('%Y%m')
res= customer.groupby('등록연월')['고객이름'].count()
res


등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

In [16]:
#가격 결측치 수정
sales_null = sales['item_price'].isnull() # 결측치만 모아서 변수에 넣기기

for i in list( sales.loc[ sales_null, 'item_name'].unique() ):
  price = sales.loc[ ( ~sales_null ) & (sales['item_name'] == i ), 'item_price'].max()
  # ~sales_null : 가격이 있는 것들
  # 가격이 있는 것들 중 item_name이 i 인 것!
  sales['item_price'].loc[ (sales_null) & (sales['item_name'] == i )] = price
sales.isnull().sum()

<ipython-input-16-c7a94583b352>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['item_price'].loc[ (sales_null) & (sales['item_name'] == i )] = price


purchase_date    0
item_name        0
item_price       0
customer_name    0
month            0
dtype: int64

In [17]:
#고객 이름을 key로 두 개의 데이터를 결합(join)
joinData = pd.merge(sales, customer, left_on='customer_name', right_on='고객이름', how='left')
# 고객 중 구매한 고객만 나오게 한다! (구매하지 않은 고객은 나오지 X)
#joinData = joinData.drop('고객이름', axis=1)
joinData



,purchase_date,item_name,item_price,customer_name,month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02,상품A,100.0,김가온,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05,상품S,1900.0,김우찬,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42,상품A,100.0,김유찬,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40,상품Z,2600.0,김재현,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 3:09,상품A,100.0,김강현,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 2:56,상품Y,2500.0,김정민,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 4:03,상품M,1300.0,김재원,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14,상품Q,1700.0,김지율,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56,상품H,800.0,김승주,201907,김승주,E시,2018-07-15,201807


In [18]:
#덤프파일 저장
joinData.to_csv('dumpData.csv', index=False)

In [19]:
joinData.pivot_table(index='month', columns='customer_name', values='item_price', aggfunc='sum', fill_value=0)
# index : 행의 이름, colums : 열의 이름
# values : 들어갈 데이터
# 고객 이름, 지역

customer_name,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
month,,,,,,,,,,,,,,,,,,,,,
201901,1000,1500,100,3700,7000,5200,1100,2200,1600,6400,...,1300,2200,1800,3500,1400,1600,5800,6600,6400,5400
201902,2500,1200,5500,2900,5700,8400,2300,5200,400,7200,...,6400,1700,7700,0,0,2200,1400,0,2500,3400
201903,6200,0,2400,500,1800,900,2300,1200,200,7000,...,3900,2600,1600,2900,2400,3700,100,600,7300,4300
201904,100,0,2100,0,1700,300,2100,9700,1800,5700,...,0,4900,2400,100,1200,2900,5000,6200,6600,800
201905,0,0,2600,2100,2400,4800,8500,5400,5400,2300,...,4300,1100,100,3700,5300,0,0,0,2400,3100
201906,6700,0,2000,0,2300,1400,8800,1500,2100,2000,...,2200,5600,2300,0,6500,3100,2700,0,0,2600
201907,5800,1900,1500,2900,900,2000,7200,800,3400,4200,...,500,1600,0,4000,0,3700,10400,2400,1200,3500


In [20]:
#집계 기간에 구매 이력이 없는 사용자 확인
away = pd.merge(sales, customer, left_on='customer_name', right_on='고객이름',how='right')
# 고객 중 구매하지 않은 고객 출력!
away[away['purchase_date'].isnull()][['고객이름','등록일']]

,고객이름,등록일
2999,김서우,2019-04-23
